In [ ]:
using IterativeSolvers, SparseArrays, LinearAlgebra, IncompleteLU
using AlgebraicMultigrid

In [1]:
using LowLevelFEM
gmsh.initialize()

gmsh.open("tarto.geo")

Info    : Reading 'tarto.geo'...
Info    : Meshing 1D...- Splitting solids                                                                                             
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (TrimmedCurve)
Info    : [ 10%] Meshing curve 6 (TrimmedCurve)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (TrimmedCurve)
Info    : [ 10%] Meshing curve 10 (TrimmedCurve)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Circle)
Info    : [ 20%] Meshing curve 13 (Circle)
Info    : [ 20%] Meshing curve 14 (Circle)
Info    : [ 20%] Meshing curve 15 (Circle)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Circle)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Circle)
Info    : [ 20%] M

In [2]:
mat = material("part")
prob = Problem([mat], bandwidth=:Metis)

Info    : METIS renumbering (nested dissection)...
Info    : Done METIS renumbering


Problem("tarto", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("part", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 27263)

In [3]:
supp1 = displacementConstraint("base", uz=0)
supp2 = displacementConstraint("hole1", ux=0, uy=0)
supp3 = displacementConstraint("perim", uz=-0.00001)

load1 = load("hole2", fy=-5)

("hole2", 0, -5, 0)

In [5]:
@time q = solveDisplacement(prob, [load1], [supp1, supp2, supp3]);

 13.261845 seconds (44.21 M allocations: 10.112 GiB, 8.97% gc time, 0.16% compilation time)


In [ ]:
@time q = solveDisplacement(prob, [load1], [supp1, supp2, supp3], condensed=false, iterative=false);

In [ ]:
@time q = solveDisplacement(prob, [load1], [supp1, supp2, supp3], condensed=true, iterative=false);

In [ ]:
@time q = solveDisplacement(prob, [load1], [supp1, supp2, supp3], condensed=true, iterative=false, ordering=false);

In [ ]:
@time q = solveDisplacement(prob, [load1], [supp1, supp2, supp3], condensed=false, iterative=true);

In [ ]:
@time q = solveDisplacement(prob, [load1], [supp1, supp2, supp3], condensed=true, iterative=true);

In [ ]:
using IncompleteLU, AlgebraicMultigrid
ml = smoothed_aggregation(K.A,
    presmoother=GaussSeidel(ForwardSweep()),
    postsmoother=GaussSeidel(BackwardSweep()),
    max_levels=10,
    max_coarse=500)     # AMG-hierarchia építés
M = aspreconditioner(ml)        # preconditioner objektum

In [ ]:
@time q = solveDisplacement(prob, [load1], [supp1, supp2, supp3], condensed=true, iterative=true, reltol=1e-3, maxiter=100);

In [ ]:
@time q = solveDisplacement(prob, [load1], [supp1, supp2, supp3], condensed=true, iterative=true, preconditioner=M, reltol=1e-3, maxiter=100);

In [ ]:
showDoFResults(q, name="displacement")

In [ ]:
showElementResults(solveStrain(q))

In [ ]:
showDoFResults(solveStrain(q))

In [ ]:
S = solveStress(q, DoFResults=true)
showDoFResults(S)

In [ ]:
S = solveStress(q)
showStressResults(S)

In [ ]:
using LinearAlgebra
λ, Q = eigen(S);

In [ ]:
λ1, λ2, λ3 = getEigenValues(λ);

In [ ]:
σ_Mohr = λ1 - λ3;

In [ ]:
σ_HMH = √(((λ1 - λ2) * (λ1 - λ2) + (λ2 - λ3) * (λ2 - λ3) + (λ3 - λ1) * (λ3 - λ1)) / 2);

In [ ]:
showElementResults(S, :sz, name="σz")
showElementResults(S, :s, name="von Mises - GMSH")
showElementResults(σ_Mohr, name="Mohr")
showElementResults(σ_HMH, name="von Mises")
showElementResults(σ_Mohr - σ_HMH, name="diff")
showDoFResults((σ_Mohr - σ_HMH) / σ_Mohr * 100, name="diff - smooth")

In [ ]:
openPostProcessor()

In [ ]:
gmsh.finalize()